In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#downloading required libraries
!pip install torch torchvision
!pip install pytorch_pretrained_bert
!pip install transformers
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 109.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from nltk.tree import Tree
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, SequentialSampler
from tqdm import tqdm
import os

In [4]:
for dirname, _, filenames in os.walk('/content/drive/MyDrive/inference_dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/drive/MyDrive/inference_dataset/train.tsv
/content/drive/MyDrive/inference_dataset/dev.tsv
/content/drive/MyDrive/inference_dataset/test.tsv
/content/drive/MyDrive/inference_dataset/README.txt


In [5]:
#load train
# Load the TSV file as a DataFrame
df = pd.read_csv('/content/drive/MyDrive/inference_dataset/train.tsv', sep='\t')

# Extract the premise and hypothesis sentences and labels
premises = []
hypotheses = []
labels = []
for i, row in df.iterrows():
    premise_parse_tree = Tree.fromstring(row['Sent1_parse'])
    hypothesis_parse_tree = Tree.fromstring(row['Sent2_parse'])
    premise = ' '.join(premise_parse_tree.leaves())
    hypothesis = ' '.join(hypothesis_parse_tree.leaves())
    label = row['Label']
    premises.append(premise)
    hypotheses.append(hypothesis)
    labels.append(label)


train = pd.DataFrame({'premise': premises, 'hypothesis': hypotheses, 'label': labels})

# Print the new DataFrame
train.head()

,premise,hypothesis,label
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competiti...,neutral
1,A person on a horse jumps over a broken down a...,"A person is at a diner , ordering an omelette .",contradiction
2,A person on a horse jumps over a broken down a...,"A person is outdoors , on a horse .",entailment
3,Children smiling and waving at camera,They are smiling at their parents,neutral
4,Children smiling and waving at camera,There are children present,entailment


In [6]:
#load test

# Load the TSV file as a DataFrame
dfte = pd.read_csv('/content/drive/MyDrive/inference_dataset/test.tsv', sep='\t')

# Extract the premise and hypothesis sentences and labels
premises = []
hypotheses = []
labels = []
for i, row in dfte.iterrows():
    premise_parse_tree = Tree.fromstring(row['Sent1_parse'])
    hypothesis_parse_tree = Tree.fromstring(row['Sent2_parse'])
    premise = ' '.join(premise_parse_tree.leaves())
    hypothesis = ' '.join(hypothesis_parse_tree.leaves())
    label = row['Label']
    premises.append(premise)
    hypotheses.append(hypothesis)
    labels.append(label)


test = pd.DataFrame({'premise': premises, 'hypothesis': hypotheses, 'label': labels})

# Print the new DataFrame
test.head()

,premise,hypothesis,label
0,This church choir sings to the masses as they ...,The church has cracks in the ceiling .,neutral
1,This church choir sings to the masses as they ...,The church is filled with song .,entailment
2,This church choir sings to the masses as they ...,A choir singing at a baseball game .,contradiction
3,"A woman with a green headscarf , blue shirt an...",The woman is young .,neutral
4,"A woman with a green headscarf , blue shirt an...",The woman is very happy .,entailment


In [7]:
#load eval

# Load the TSV file as a DataFrame
dfev = pd.read_csv('/content/drive/MyDrive/inference_dataset/dev.tsv', sep='\t')

# Extract the premise and hypothesis sentences and labels
premises = []
hypotheses = []
labels = []
for i, row in dfev.iterrows():
    premise_parse_tree = Tree.fromstring(row['Sent1_parse'])
    hypothesis_parse_tree = Tree.fromstring(row['Sent2_parse'])
    premise = ' '.join(premise_parse_tree.leaves())
    hypothesis = ' '.join(hypothesis_parse_tree.leaves())
    label = row['Label']
    premises.append(premise)
    hypotheses.append(hypothesis)
    labels.append(label)


eval = pd.DataFrame({'premise': premises, 'hypothesis': hypotheses, 'label': labels})

# Print the new DataFrame
train.head()

,premise,hypothesis,label
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competiti...,neutral
1,A person on a horse jumps over a broken down a...,"A person is at a diner , ordering an omelette .",contradiction
2,A person on a horse jumps over a broken down a...,"A person is outdoors , on a horse .",entailment
3,Children smiling and waving at camera,They are smiling at their parents,neutral
4,Children smiling and waving at camera,There are children present,entailment


In [8]:
#check for lable entries
print(train['label'].unique())
print(eval['label'].unique())
print(test['label'].unique())

['neutral' 'contradiction' 'entailment' '-']
['neutral' 'entailment' 'contradiction' '-']
['neutral' 'entailment' 'contradiction' '-']


In [9]:
#removing - from lables
train_pro = train[train['label'] != '-']

test_pro = test[test['label'] != '-']

eval_pro = eval[eval['label'] != '-']

In [10]:
train_pro['label'] = train_pro['label'].replace('neutral', 0).replace('entailment', 1).replace('contradiction', 2)

<ipython-input-10-8cf82afbfc1d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_pro['label'] = train_pro['label'].replace('neutral', 0).replace('entailment', 1).replace('contradiction', 2)


In [11]:
test_pro['label'] = test_pro['label'].replace('neutral', 0).replace('entailment', 1).replace('contradiction', 2)

<ipython-input-11-1db7df12dd9c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_pro['label'] = test_pro['label'].replace('neutral', 0).replace('entailment', 1).replace('contradiction', 2)


In [12]:
eval_pro['label'] = eval_pro['label'].replace('neutral', 0).replace('entailment', 1).replace('contradiction', 2)
eval_pro.head()

<ipython-input-12-542ae9af0cc6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_pro['label'] = eval_pro['label'].replace('neutral', 0).replace('entailment', 1).replace('contradiction', 2)


,premise,hypothesis,label
0,Two women are embracing while holding to go pa...,The sisters are hugging goodbye while holding ...,0
1,Two women are embracing while holding to go pa...,Two woman are holding packages .,1
2,Two women are embracing while holding to go pa...,The men are fighting outside a deli .,2
3,"Two young children in blue jerseys , one with ...",Two kids in numbered jerseys wash their hands .,1
4,"Two young children in blue jerseys , one with ...",Two kids at a ballgame wash their hands .,0


In [13]:
# Minimize the datasets for quick trials
train_pro = train_pro.iloc[:50000, :]
eval_pro = eval_pro.sample(100)
test_pro = test_pro.sample(100)

train_pro.head(3)

,premise,hypothesis,label
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competiti...,0
1,A person on a horse jumps over a broken down a...,"A person is at a diner , ordering an omelette .",2
2,A person on a horse jumps over a broken down a...,"A person is outdoors , on a horse .",1


In [14]:
print(train_pro.isnull().sum())
print(eval_pro.isnull().sum())
print(test_pro.isnull().sum())

premise       0
hypothesis    0
label         0
dtype: int64
premise       0
hypothesis    0
label         0
dtype: int64
premise       0
hypothesis    0
label         0
dtype: int64


In [15]:
#counting for url entries in the datasets
print(train_pro['premise'][train_pro['premise'].str.contains('http')].count())
print(train_pro['hypothesis'][train_pro['hypothesis'].str.contains('http')].count())

print(eval_pro['premise'][eval_pro['premise'].str.contains('http')].count())
print(eval_pro['hypothesis'][eval_pro['hypothesis'].str.contains('http')].count())

print(test_pro['premise'][test_pro['premise'].str.contains('http')].count())
print(test_pro['hypothesis'][test_pro['hypothesis'].str.contains('http')].count())


0
0
0
0
0
0


In [16]:
# copy dataset 
train_df = train_pro
test_df = test_pro
eval_df = eval_pro

In [17]:
# Split the training set into train and validation sets
train_df, val_df = train_test_split(train_df, test_size=0.2)

In [18]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [19]:
#implementing tokenizer
train_encodings = tokenizer(train_df['premise'].tolist(), train_df['hypothesis'].tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_df['premise'].tolist(), val_df['hypothesis'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_df['premise'].tolist(), test_df['hypothesis'].tolist(), truncation=True, padding=True)
eval_encodings = tokenizer(eval_df['premise'].tolist(), eval_df['hypothesis'].tolist(), truncation=True, padding=True)

In [20]:

#seaprating lables
train_labels = train_df['label']
val_labels = val_df['label']
test_labels = test_df['label']
eval_labels = eval_df['label']

In [21]:
# Convert the labels to PyTorch tensors
train_labels = torch.tensor(train_df['label'].values)
val_labels = torch.tensor(val_df['label'].values)
test_labels = torch.tensor(test_df['label'].values)
eval_labels = torch.tensor(eval_df['label'].values)

In [22]:
# Convert the encodings and labels to PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels)
)
val_dataset = torch.utils.data.TensorDataset(
    torch.tensor(val_encodings['input_ids']),
    torch.tensor(val_encodings['attention_mask']),
    torch.tensor(val_labels)
)
test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_labels)
)
eval_dataset = torch.utils.data.TensorDataset(
    torch.tensor(eval_encodings['input_ids']),
    torch.tensor(eval_encodings['attention_mask']),
    torch.tensor(eval_labels)
)

<ipython-input-22-c03db9aa780a>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(train_labels)
<ipython-input-22-c03db9aa780a>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(val_labels)
<ipython-input-22-c03db9aa780a>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(test_labels)
<ipython-input-22-c03db9aa780a>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sour

In [23]:
# Load the BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [24]:
# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [25]:
# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [26]:
# Set up the data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)
eval_loader = torch.utils.data.DataLoader(eval_dataset, batch_size=32, shuffle=False)


In [27]:
# Train the model
num_epochs = 3
for epoch in range(num_epochs):
    # Train for one epoch
    model.train()
    train_loss = 0.0
    for batch_idx, batch in enumerate(train_loader):
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * batch[0].size(0)

        if batch_idx % 100 == 0:
            print('Epoch [{}/{}], Batch [{}/{}], Training Loss: {:.4f}'.format(
                epoch+1, num_epochs, batch_idx+1, len(train_loader), train_loss/(batch_idx+1)))

    train_loss /= len(train_loader.dataset)
    print('Epoch [{}/{}], Training Loss: {:.4f}'.format(epoch+1, num_epochs, train_loss))

Epoch [1/3], Batch [1/1250], Training Loss: 38.4525
Epoch [1/3], Batch [101/1250], Training Loss: 30.5418
Epoch [1/3], Batch [201/1250], Training Loss: 26.6029
Epoch [1/3], Batch [301/1250], Training Loss: 24.2334
Epoch [1/3], Batch [401/1250], Training Loss: 22.9889
Epoch [1/3], Batch [501/1250], Training Loss: 22.1501
Epoch [1/3], Batch [601/1250], Training Loss: 21.4211
Epoch [1/3], Batch [701/1250], Training Loss: 20.9107
Epoch [1/3], Batch [801/1250], Training Loss: 20.4751
Epoch [1/3], Batch [901/1250], Training Loss: 20.1556
Epoch [1/3], Batch [1001/1250], Training Loss: 19.8397
Epoch [1/3], Batch [1101/1250], Training Loss: 19.5127
Epoch [1/3], Batch [1201/1250], Training Loss: 19.2157
Epoch [1/3], Training Loss: 0.5967
Epoch [2/3], Batch [1/1250], Training Loss: 12.5057
Epoch [2/3], Batch [101/1250], Training Loss: 11.2425
Epoch [2/3], Batch [201/1250], Training Loss: 11.3141
Epoch [2/3], Batch [301/1250], Training Loss: 11.6870
Epoch [2/3], Batch [401/1250], Training Loss: 11

In [28]:
# Evaluate the model on the validation set
val_losses = []
val_correct = 0
val_total = 0
val_predictions = []
val_targets = []

model.eval()
for batch in tqdm(val_loader, desc='Evaluating on validation set'):
    inputs = {'input_ids': batch[0].to(device), 'attention_mask': batch[1].to(device), 'labels': batch[2].to(device)}
    with torch.no_grad():
        outputs = model(**inputs)
        loss = outputs.loss
        logits = outputs.logits

    val_losses.append(loss.item())

    predictions = torch.argmax(logits, dim=1).tolist()
    targets = batch[2].tolist()

    val_correct += sum([1 if p == t else 0 for p, t in zip(predictions, targets)])
    val_total += len(predictions)

    val_predictions += predictions
    val_targets += targets

val_accuracy = val_correct / val_total
val_loss = sum(val_losses) / len(val_losses)
print(f'Validation accuracy: {val_accuracy:.4f}, validation loss: {val_loss:.4f}')

Evaluating on validation set: 100%|██████████| 313/313 [00:54<00:00,  5.79it/s]

Validation accuracy: 0.8209, validation loss: 0.5827


In [29]:
# Save the validation and test predictions to .tsv files
val_output_df = pd.DataFrame({'premise': val_df['premise'], 'hypothesis': val_df['hypothesis'], 'predicted_label': val_predictions, 'true_label':val_targets})


val_output_df.to_csv('val_predictions.tsv', sep='\t', index=False)


In [30]:
# Evaluate the model on the test set
test_losses = []
test_correct = 0
test_total = 0
test_predictions = []
test_targets = []

model.eval()
for batch in tqdm(test_loader, desc='Evaluating on test set'):
    inputs = {'input_ids': batch[0].to(device), 'attention_mask': batch[1].to(device), 'labels': batch[2].to(device)}
    with torch.no_grad():
        outputs = model(**inputs)
        loss = outputs.loss
        logits = outputs.logits

    test_losses.append(loss.item())

    predictions = torch.argmax(logits, dim=1).tolist()
    targets = batch[2].tolist()

    test_correct += sum([1 if p == t else 0 for p, t in zip(predictions, targets)])
    test_total += len(predictions)

    test_predictions += predictions
    test_targets += targets

test_accuracy = test_correct / test_total
test_loss = sum(test_losses) / len(test_losses)
print(f'Test accuracy: {test_accuracy:.4f}, test loss: {test_loss:.4f}')

Evaluating on test set: 100%|██████████| 4/4 [00:00<00:00,  7.88it/s]

Test accuracy: 0.8200, test loss: 0.5857


In [31]:
# Save the test predictions to a .tsv file
test_output_df = pd.DataFrame({'premise': test_df['premise'], 'hypothesis': test_df['hypothesis'], 'predicted_label': test_predictions, 'true_label':test_targets})

test_output_df.to_csv('test_predictions.tsv', sep='\t', index=False)

In [33]:
from sklearn.metrics import accuracy_score, f1_score

# Convert the predictions and targets to numpy arrays
test_predictions = np.array(test_predictions)
test_targets = np.array(test_targets)

# Calculate the accuracy and F1 score
accuracy = accuracy_score(test_targets, test_predictions)
f1 = f1_score(test_targets, test_predictions, average='weighted')

print(f'Test accuracy: {accuracy:.4f}')
print(f'Test F1 score: {f1:.4f}')


Test accuracy: 0.8200
Test F1 score: 0.8189
